In [ ]:
"""
Población por sector de actividad económica (México, Nacional)
- Lee 3 indicadores % (Primario, Secundario, Terciario) del BIE
- Lee población ocupada total (denominador) del BIE
- Calcula valores absolutos y genera PNG/HTML y CSV
"""

import os
import json
import csv
import requests
import plotly.graph_objects as go
import streamlit as st # <-- ¡IMPORTACIÓN DE STREAMLIT AÑADIDA!

# =========================
# 🔧 CONFIGURACIÓN
# =========================
TOKEN = "460ccba7-40b7-08a2-47dd-7301e6b6fbbc"
ID_PRIMARIO = 668908
ID_SECUNDARIO = 668906
ID_TERCIARIO= 668912
ID_OCUPADOS = 289242

# Estilos que se mantendrán fijos
COLOR_LABEL = "#ffffff"
COLOR_GRID = "#e5e7eb"
COLOR_EJES = "#6b7280"

# Salidas
PNG = "poblacion_sector.png"
HTML = "poblacion_sector.html"
CSVF = "poblacion_sector.csv"

# ---------------------------------------------
# Adaptación de Estilo para Streamlit
# ---------------------------------------------
DEFAULT_FONT = "Aptos Light, Arial, sans-serif"
# Orden de colores por defecto: [Barra, Anotación_Percentaje]
DEFAULT_PALETTE = ["#0e1a2b", "#ff9f18"]

# Obtener variables inyectadas de Streamlit (si existen) o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Asignar variables de estilo que se usan en la gráfica Plotly
FUENTE = FONT
COLOR_BARRA = PALETTE[0] if len(PALETTE) > 0 else DEFAULT_PALETTE[0]
AMARILLO = PALETTE[1] if len(PALETTE) > 1 else DEFAULT_PALETTE[1] # Acento para el porcentaje

# (El VERDE original no se usa en el cuerpo de la gráfica, solo como acento no utilizado:
# VERDE = PALETTE[2] if len(PALETTE) > 2 else "#889064")

def bie_url(ids: str) -> str:
    return (
        "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/"
        f"INDICATOR/{ids}/es/00/true/BIE-BISE/2.0/{TOKEN}?type=json"
    )

def fetch_bie_latest(ind_id: int):
    r = requests.get(bie_url(str(ind_id)), timeout=30)
    r.raise_for_status()
    j = r.json()
    ser = j["Series"][0]
    obs = ser["OBSERVATIONS"][0]
    valor = float(obs["OBS_VALUE"])
    periodo = obs["TIME_PERIOD"]         # p.ej. "2025/09"
    unidad = ser.get("UNIT", "")
    return valor, periodo, unidad, ser

def formato_personas(n: float) -> str:
    return f"{round(n):,}"

# -------------------------
# 1) Lee porcentajes
# -------------------------
p_prim, periodo_a, _, _ = fetch_bie_latest(ID_PRIMARIO)
p_sec, periodo_b, _, _ = fetch_bie_latest(ID_SECUNDARIO)
p_terc, periodo_c, _, _ = fetch_bie_latest(ID_TERCIARIO)

# Alineación de periodos: tomamos el más reciente común (asumiendo misma fecha)
periodo = periodo_a # si quieres, valida que los 3 coincidan y, si no, decide la intersección

# Normalización suave si por redondeos no cierran exactamente en 100
suma = p_prim + p_sec + p_terc
if abs(suma - 100) > 0.5:
    # Advertencia mínima; si supera 0.5 pp, normaliza
    p_prim = p_prim * (100.0 / suma)
    p_sec = p_sec * (100.0 / suma)
    p_terc = p_terc * (100.0 / suma)

# -------------------------
# 2) Denominador: ocupados
# -------------------------
ocupados, periodo_den, unidad_den, meta_den = fetch_bie_latest(ID_OCUPADOS)

# ⚠️ Validación de unidad:
# - Si el indicador viene en millones, el metadato suele indicarlo; ajusta a personas.
unidad_den_lower = str(unidad_den).lower()
if "mill" in unidad_den_lower: # "millones", "million"
    ocupados = ocupados * 1_000_000

# -------------------------
# 3) Cálculo de absolutos
# -------------------------
v_prim = ocupados * (p_prim / 100.0)
v_sec = ocupados * (p_sec / 100.0)
v_terc = ocupados * (p_terc / 100.0)

categorias = ["Primaria", "Secundaria", "Terciaria"]
valores = [v_prim, v_sec, v_terc]
porcents = [p_prim, p_sec, p_terc]

# -------------------------
# 4) Gráfica Plotly
# -------------------------
titulo = "Población por sector de actividad económica"
subtitulo = f"(Nacional • Población ocupada • {periodo})"

fig = go.Figure()
fig.add_bar(
    x=categorias,
    y=valores,
    marker_color=COLOR_BARRA, # <-- Color dinámico de Streamlit
    text=[formato_personas(v) for v in valores],
    textposition="inside",
    textfont=dict(color=COLOR_LABEL, size=14),
)

fig.update_layout(
    title=dict(text=f"{titulo}<br><sup>{subtitulo}</sup>", x=0.5),
    font=dict(family=FUENTE, size=16, color="#111827"), # <-- Fuente dinámica de Streamlit
    plot_bgcolor="white", paper_bgcolor="white",
    showlegend=False,
    margin=dict(l=40, r=40, t=80, b=60),
)

fig.update_xaxes(title_text="", tickfont=dict(size=15))
fig.update_yaxes(
    title_text="Personas",
    gridcolor=COLOR_GRID, zeroline=False,
    tickfont=dict(size=13, color=COLOR_EJES),
    tickformat=",.0f",
    separatethousands=True
)

# Acentos sutiles acorde a tu paleta (anotaciones %)
for x, y, p in zip(categorias, valores, porcents):
    fig.add_annotation(
        x=x, y=y, text=f"{p:.2f}%",
        showarrow=False, yshift=18,
        font=dict(color=AMARILLO, size=13, family=FUENTE) # <-- Color/Fuente dinámicos
    )

# Exporta (Estas líneas se ejecutan si el script no está en el entorno Streamlit, o si quieres los archivos)
#fig.write_image(PNG, scale=3, width=1280, height=720)
#fig.write_html(HTML, include_plotlyjs="cdn")
'''
with open(CSVF, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["Sector", "Porcentaje", "Personas", "Periodo_porcentajes", "Periodo_ocupados", "Unidad_ocupados"])
    for c, p, v in zip(categorias, porcents, valores):
        w.writerow([c, f"{p:.6f}", round(v), periodo, periodo_den, unidad_den])

print(f"Listo:\n- {PNG}\n- {HTML}\n- {CSVF}")
'''
# fig.show()

# -------------------------
# 5) Muestra en Streamlit
# -------------------------
st.plotly_chart(fig, use_container_width=True) # <-- ¡LÍNEA CLAVE AÑADIDA!